In [1]:
%pip install tensorflow

In [72]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [73]:
# read the data
#-------------------
data = pd.read_csv("final_data.csv")
data.describe()
data = data[data["mag"] >= 3]

In [74]:
data.describe()

,latitude,longitude,mag,depth,timestamp
count,664875.000000,664875.000000,664875.000000,664875.000000,6.648750e+05
mean,13.403848,9.950133,4.214894,66.061349,1.050162e+09
std,30.206160,120.224505,0.694671,111.978909,3.944005e+08
min,-84.422000,-179.999000,3.000000,0.000000,9.472320e+07
25%,-9.260000,-97.258000,3.700000,10.000000,7.903134e+08
50%,17.368000,21.795000,4.300000,33.000000,1.113404e+09
75%,38.000000,128.434000,4.700000,60.300000,1.377632e+09
max,87.386000,180.000000,9.100000,735.800000,1.641010e+09


Training Data

In [75]:
from sklearn.model_selection import train_test_split

X = data[["latitude", "longitude", "timestamp"]]
y = data[["mag", "depth"]]
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(664875, 3) (664875, 2)
(531900, 3) (531900, 2) (132975, 3) (132975, 2)


In [76]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(X_train)

In [77]:
def get_basic_model():
	model = tf.keras.Sequential([
		normalizer,
		tf.keras.layers.Dense(16, activation='relu', input_shape=(3,)),
		tf.keras.layers.Dense(16, activation='relu'),
		tf.keras.layers.Dense(2, activation='softmax')
	])
	model.compile(optimizer='adam',
			loss=tf.keras.losses.SquaredHinge(),
			metrics=['accuracy'])
	return model

In [78]:
model = get_basic_model()
history = model.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test))

Epoch 1/15
16622/16622 [==============================] - 60s 4ms/step - loss: 0.0061 - accuracy: 0.9547 - val_loss: 0.0062 - val_accuracy: 0.9667
Epoch 2/15
16622/16622 [==============================] - 46s 3ms/step - loss: 0.0061 - accuracy: 0.9667 - val_loss: 0.0062 - val_accuracy: 0.9667
Epoch 3/15
16622/16622 [==============================] - 66s 4ms/step - loss: 0.0061 - accuracy: 0.9667 - val_loss: 0.0062 - val_accuracy: 0.9667
Epoch 4/15
16622/16622 [==============================] - 98s 6ms/step - loss: 0.0061 - accuracy: 0.9667 - val_loss: 0.0062 - val_accuracy: 0.9667
Epoch 5/15
16621/16622 [============================>.] - ETA: 0s - loss: 0.0061 - accuracy: 0.9667

In [66]:
history.history

{'loss': [0.01245432160794735,
  0.004546043928712606,
  0.004537333268672228,
  0.004535944666713476,
  0.004535396117717028,
  0.004535133484750986,
  0.004535007290542126,
  0.004534891806542873,
  0.004534792620688677,
  0.004534732084721327,
  0.004534699022769928,
  0.004534689709544182,
  0.004534686915576458,
  0.004534686915576458,
  0.004534686915576458],
 'accuracy': [0.8218199014663696,
  0.9666779637336731,
  0.9666779637336731,
  0.9666779637336731,
  0.9666779637336731,
  0.9666779637336731,
  0.9666779637336731,
  0.9666779637336731,
  0.9666779637336731,
  0.9666779637336731,
  0.9666779637336731,
  0.9666779637336731,
  0.9666779637336731,
  0.9666779637336731,
  0.9666779637336731],
 'val_loss': [0.00481219869107008,
  0.004764414392411709,
  0.004762360360473394,
  0.004761787131428719,
  0.0047612497583031654,
  0.0047609843313694,
  0.004760760813951492,
  0.004760551266372204,
  0.004760398529469967,
  0.004760309122502804,
  0.00476029422134161,
  0.004760293290

In [92]:
%pip install pyyaml h5py


   ---------------------------------------- 0.0/144.7 kB ? eta -:--:--
   -- ------------------------------------- 10.2/144.7 kB ? eta -:--:--
   ---------------------------------------  143.4/144.7 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 144.7/144.7 kB 1.4 MB/s eta 0:00:00


In [ ]:

# Save the entire model as a `.keras` zip archive.
model.save('neural_network.tf')

INFO:tensorflow:Assets written to: neural_network.tf\assets


INFO:tensorflow:Assets written to: neural_network.tf\assets


In [ ]:
new_model = tf.keras.models.load_model('neural_network.tf')

In [ ]:
new_model.evaluate(X_test[0:100], y_test[0:100], verbose=2)

4/4 - 0s - loss: 0.0000e+00 - accuracy: 0.9500 - 143ms/epoch - 36ms/step


[0.0, 0.949999988079071]

In [ ]:
new_model.predict(X_test[0:100])

4/4 [==============================] - 0s 3ms/step


array([[6.16071663e+01, 1.41192749e+03],
       [2.70113583e+01, 6.10643982e+02],
       [7.61571550e+00, 1.66689301e+02],
       [1.27842629e+00, 2.19345036e+01],
       [1.70928879e+01, 3.94190094e+02],
       [1.98770964e+00, 3.34694595e+01],
       [1.80576122e+00, 3.35611839e+01],
       [6.05160332e+01, 1.39831042e+03],
       [2.71776104e+01, 6.18879395e+02],
       [6.67874298e+01, 1.54932446e+03],
       [2.69461250e+01, 6.16596436e+02],
       [1.59364395e+01, 3.66597565e+02],
       [2.90984383e+01, 6.72051025e+02],
       [4.59023476e+00, 9.50783081e+01],
       [2.79741516e+01, 6.49483643e+02],
       [2.84800072e+01, 6.56190430e+02],
       [2.37396088e+01, 5.42027222e+02],
       [1.43008304e+00, 2.42540588e+01],
       [7.25019789e+00, 1.53721619e+02],
       [1.57571840e+01, 3.62236786e+02],
       [1.57199776e+00, 2.58358288e+01],
       [6.17936668e+01, 1.43044397e+03],
       [1.44031894e+00, 2.34010258e+01],
       [3.76631546e+01, 8.67556213e+02],
       [3.782347

In [ ]:
y_test[0:100]

,mag,depth
171832,4.7,33.0
77284,3.8,120.0
693662,5.2,33.0
1611710,4.4,10.0
3068472,4.3,185.9
...,...,...
917786,3.3,124.8
2997505,5.2,10.0
650055,3.6,10.0
1571005,3.2,2.0


In [ ]:
new_model.evaluate(X_test[0:100], y_test[0:100], verbose=2)


4/4 - 1s - loss: 1.0745e-04 - accuracy: 0.9500 - 661ms/epoch - 165ms/step


[0.00010745367762865499, 0.949999988079071]